<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/344_WDO_Risk_Node.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Workforce Development Orchestrator — Automation Risk Analysis Node

The `automation_risk_analysis_node` is responsible for **executing automation risk assessment in a controlled, policy-driven way**. Rather than embedding scoring logic directly in the node, it acts as a coordinator that applies previously tested utilities to the current workforce state.

This separation ensures that **analysis remains consistent, explainable, and governable**.

---

## 1. Clear Contract With State

The node explicitly declares the data it depends on:

* Roles
* Tasks grouped by role
* Employees grouped by role
* Employee roster

If any required input is missing, the node exits early with a clear error message.

This prevents:

* Partial or misleading analysis
* Silent degradation
* Downstream nodes operating on incomplete intelligence

From a governance perspective, this establishes a **hard precondition** for automation risk conclusions.

---

## 2. Role-Level Risk Is the Primary Lens

The node first performs **role-level automation risk analysis**.

This reflects how real organizations reason about work:

* Roles define responsibilities
* Tasks define exposure
* People inherit risk through role structure

By analyzing roles first, the system produces insights that are:

* Strategically meaningful
* Suitable for workforce planning
* Comparable across teams and departments

---

## 3. Employee-Level Risk Is Derived, Not Invented

Employee analysis is computed by mapping individuals to their role’s task risk profile.

This design ensures:

* No conflicting narratives between role and employee risk
* No “personalized” risk invented without justification
* Ethical consistency across workforce communications

Employees are affected *by work structure*, not judged independently.

---

## 4. Thin Node, Tested Logic

The node itself:

* Contains no scoring logic
* Contains no thresholds
* Contains no recommendations

All of that responsibility lives in utilities that were:

* Independently tested
* Configuration-driven
* Reusable

This keeps orchestration readable and trustworthy.

---

## 5. Failure Is Explicit and Contained

Unexpected errors are caught and recorded in state rather than crashing execution.

This allows:

* Partial reports with clear caveats
* Debugging without reruns
* Post-run diagnostics for leadership or auditors

Again, transparency is prioritized over silent failure.

---

## Why This Design Builds Trust

From a leadership standpoint, this node guarantees that:

* Automation risk is computed consistently
* No hidden intelligence is introduced at orchestration time
* Every conclusion can be traced back to policy and data

From an engineering standpoint, it guarantees:

* Predictable behavior
* Easy testing
* Clear extension points

---

## Architectural Takeaway

This node demonstrates an essential orchestration principle:

> **Nodes decide when to act; utilities decide how to think.**

By keeping automation risk logic outside the node and applying it deterministically, the orchestrator earns credibility when it later recommends reskilling or role evolution.



In [ ]:
def automation_risk_analysis_node(
    state: WorkforceDevelopmentOrchestratorState,
    config: WorkforceDevelopmentOrchestratorConfig
) -> Dict[str, Any]:
    """
    Automation Risk Analysis Node: Orchestrate analyzing automation risk.

    Analyzes automation risk for all roles and employees based on their tasks.
    """
    errors = state.get("errors", [])

    # Get required data from state
    roles = state.get("roles", [])
    tasks_by_role = state.get("tasks_by_role", {})
    employees_by_role = state.get("employees_by_role", {})
    employees = state.get("employees", [])

    if not roles:
        return {
            "errors": errors + ["automation_risk_analysis_node: roles data required"]
        }

    if not tasks_by_role:
        return {
            "errors": errors + ["automation_risk_analysis_node: tasks_by_role data required"]
        }

    try:
        # Analyze all roles
        role_analyses = analyze_all_roles_automation_risk(
            roles,
            tasks_by_role,
            employees_by_role,
            config
        )

        # Analyze all employees
        employee_analyses = analyze_all_employees_automation_risk(
            employees,
            tasks_by_role,
            config
        )

        return {
            "automation_risk_analysis": role_analyses,  # Store role-level analyses
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"automation_risk_analysis_node: Unexpected error: {str(e)}"]
        }



# Test automation risk analysis node

In [ ]:
"""Test automation risk analysis node

Testing Phase 3: Automation Risk Analysis Node
Following the pattern: Test node after utilities pass
"""

from agents.workforce_development_orchestrator.nodes import (
    goal_node,
    planning_node,
    data_loading_node,
    automation_risk_analysis_node
)
from config import (
    WorkforceDevelopmentOrchestratorState,
    WorkforceDevelopmentOrchestratorConfig
)


def test_automation_risk_analysis_node():
    """Test automation risk analysis node"""
    state: WorkforceDevelopmentOrchestratorState = {
        "employee_id": None,
        "errors": []
    }
    config = WorkforceDevelopmentOrchestratorConfig()

    # First load data
    state = goal_node(state)
    state = planning_node(state)
    state = data_loading_node(state, config)

    # Then analyze automation risk
    result = automation_risk_analysis_node(state, config)

    # Check that analysis is present
    assert "automation_risk_analysis" in result
    assert len(result["automation_risk_analysis"]) > 0

    # Check structure of first analysis
    first_analysis = result["automation_risk_analysis"][0]
    assert "role_id" in first_analysis
    assert "overall_risk_score" in first_analysis
    assert "risk_level" in first_analysis
    assert "high_risk_tasks" in first_analysis
    assert "affected_employees" in first_analysis
    assert "recommendations" in first_analysis

    # Verify no errors
    assert len(result.get("errors", [])) == 0

    print("✅ test_automation_risk_analysis_node: PASSED")


def test_automation_risk_analysis_node_requires_data():
    """Test automation risk analysis node requires data"""
    state: WorkforceDevelopmentOrchestratorState = {
        "errors": []
    }
    config = WorkforceDevelopmentOrchestratorConfig()

    result = automation_risk_analysis_node(state, config)

    # Should have errors
    assert "errors" in result
    assert len(result["errors"]) > 0

    print("✅ test_automation_risk_analysis_node_requires_data: PASSED")


def test_automation_risk_analysis_integration():
    """Test automation risk analysis with full workflow"""
    state: WorkforceDevelopmentOrchestratorState = {
        "employee_id": None,
        "errors": []
    }
    config = WorkforceDevelopmentOrchestratorConfig()

    # Full workflow up to automation risk analysis
    state = goal_node(state)
    state = planning_node(state)
    state = data_loading_node(state, config)
    state = automation_risk_analysis_node(state, config)

    # Verify all data is present
    assert "employees" in state
    assert "roles" in state
    assert "automation_risk_analysis" in state
    assert len(state["automation_risk_analysis"]) == 5  # 5 roles

    # Verify analysis quality
    for analysis in state["automation_risk_analysis"]:
        assert 0.0 <= analysis["overall_risk_score"] <= 1.0
        assert analysis["risk_level"] in ["high", "medium", "low"]

    print("✅ test_automation_risk_analysis_integration: PASSED")


if __name__ == "__main__":
    print("=" * 60)
    print("Testing Automation Risk Analysis Node (Phase 3)")
    print("=" * 60)
    print()

    test_automation_risk_analysis_node()
    test_automation_risk_analysis_node_requires_data()
    test_automation_risk_analysis_integration()

    print()
    print("=" * 60)
    print("✅ All automation risk node tests passed!")
    print("=" * 60)



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_008_Workforce_Development_Orchestrator % python3 test_automation_risk_node.py
============================================================
Testing Automation Risk Analysis Node (Phase 3)
============================================================

✅ test_automation_risk_analysis_node: PASSED
✅ test_automation_risk_analysis_node_requires_data: PASSED
✅ test_automation_risk_analysis_integration: PASSED

============================================================
✅ All automation risk node tests passed!
============================================================
